In [ ]:
import numpy as np 
import pandas as pd 
import csv
import numpy
import pickle
from collections import Counter
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston
import xgboost as xgb

read data

In [ ]:
orders     = pd.read_csv("../capstone_data/orders.csv")
prior      = pd.read_csv("../capstone_data/order_products__prior.csv")
train      = pd.read_csv("../capstone_data/order_products__train.csv")
sample_sub = pd.read_csv("../capstone_data/sample_submission.csv") 

# Data Purpose

1. prior : develop model
2. train : validation
3. test : for submission

In [5]:
Counter(orders['eval_set'])

Counter({'prior': 3214874, 'test': 75000, 'train': 131209})

### prior

In [44]:
# training set : 3000만개
print("total number of orders : " + str(len(prior.order_id.unique())))
print("total number of sold product in orders : " + str(prior.shape[0]))
prior.head()

total number of orders : 3214874
total number of sold product in orders : 32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### train

In [7]:
# train set : 100만개
print("total number of orders : " + str(len(train.order_id.unique())))
print("total number of sold product in orders : " + str(train.shape[0]))
train.head()

total number of orders : 131209
total number of sold product in orders : 1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


### test

In [22]:
# test set : 100만개
test = orders.loc[orders['eval_set'] == 'test']
print("total number of orders : " + str(len(test.order_id.unique())))
print("total number of sold product in orders : " + str(test.shape[0]))

total number of orders : 75000
total number of sold product in orders : 75000


In [40]:
# test set의 구성
orders.loc[orders['eval_set'] == 'test'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


# train 과  test에 대한 이해

* prior은 모델 구축에 사용하는 training data로 사용되는 것 이다

evaluation에는 eval_set에서 train과 test를 사용할 수 있다

이 데이터는 한 유저에 대한 기록들이 존재하는데

prior이 쭉 주어지다가 마지막 것이 train(user1의 경우) 또는 test(user3의 경우)로 들어감

In [41]:
orders.loc[orders['user_id']==1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [42]:
orders.loc[orders['user_id']==3]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
26,1374495,3,prior,1,1,14,NaN
27,444309,3,prior,2,3,19,9.0
28,3002854,3,prior,3,3,16,21.0
29,2037211,3,prior,4,2,18,20.0
30,2710558,3,prior,5,0,17,12.0
31,1972919,3,prior,6,0,16,7.0
32,1839752,3,prior,7,0,15,7.0
33,3225766,3,prior,8,0,17,7.0
34,3160850,3,prior,9,0,16,7.0
35,676467,3,prior,10,3,16,17.0


## Submission

In [ ]:
sample_sub